# Phys 41 Homework 1 Jake Anderson 1/11/2024

## Problem 1: Basic numpy

In [1]:
import numpy as np

## Problem 1: Basic numpy

In [6]:
def wave(t, omega, n, phi):
    return np.sin((omega * t) ** n + phi)

In [13]:
def quadratic(a, b, c):
    sol1 = (-1 * b + np.sqrt(b**2 - 4 * a * c + 0j)) / (2 * a)
    sol2 = (-1 * b - np.sqrt(b**2 - 4 * a * c + 0j)) / (2 * a)
    return (sol1, sol2)

In [40]:
def grid_function(grid_points):
    x, y, z = grid_points
    return x * y**2 + z

In [99]:
x = np.linspace(-1, 1, 10)
y = np.linspace(-2, 2, 10)
z = np.linspace(-3, 3, 10)

grid_points = np.meshgrid(x, y, z)
result = grid_function(grid_points)
print(result.shape)

# For a specific point (x, y, z) on the grid, we can get the result there like this:
point = [1, 2, -1]
x_index = np.where(x == point[0])[0][0] # The 2 trailing [0]'s are to get it to an integer
y_index = np.where(y == point[1])[0][0]
z_index = np.where(z == point[2])[0][0]
print(result[x_index][y_index][z_index])

(10, 10, 10)
3.0
